In [ ]:
import os, sys
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')

sys.path.insert(0, os.path.abspath('..'))

from dotenv import load_dotenv
load_dotenv('../.env')

print('✅ Ready' if os.getenv('GROQ_API_KEY') else '❌ GROQ_API_KEY not found')

In [ ]:
from langchain_core.messages import HumanMessage
from src.state import AgentState
from src.graph import create_graph
from src.tools.financial_tools import calculate_rsi, calculate_macd, calculate_sma

print('✅ Modules loaded')

## Test Tools

In [ ]:
import yfinance as yf

TICKER = "AAPL"
df = yf.Ticker(TICKER).history(period="3mo")

df['SMA_20'] = calculate_sma(df)
df['RSI_14'] = calculate_rsi(df)

last = df.iloc[-1]
print(f"Price: ${last['Close']:.2f} | SMA20: ${last['SMA_20']:.2f} | RSI: {last['RSI_14']:.1f}")

## Run Analysis

In [ ]:
def analyze(ticker: str) -> dict:
    print(f"\n🚀 Analyzing {ticker}...")
    graph = create_graph()
    
    state: AgentState = {
        "messages": [HumanMessage(content=f"Analyze stock {ticker}")],
        "next": "Supervisor",
        "current_ticker": ticker,
        "current_stock_data": {},
        "iteration_count": 0,
    }
    
    result = None
    for event in graph.stream(state, config={"recursion_limit": 25}):
        for node, output in event.items():
            print(f"  ✓ {node}")
            result = output
    
    print("✅ Done")
    return result

In [ ]:
result = analyze("AAPL")

In [ ]:
# Display report
if result and "messages" in result:
    for msg in result["messages"]:
        if "[ANALYST" in msg.content:
            display(Markdown(msg.content.replace("[ANALYST - FINAL REPORT]\n\n", "")))
            break